In [4]:
import pandas as pd
import numpy as np
import sklearn
import random
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [5]:
#I don't think you guys would need to run this at all
import os
os.getcwd()
os.chdir('d:\\Harris Course work\\MLPP\\ml_proj\\obamacare-ml-project')

In [6]:
#ChatGPT support
# set the file path and chunk size
file_path = 'data/yelp_academic_dataset_review.json'
chunk_size = 1000

# determine the total number of records in the file
with open(file_path, 'r', encoding='utf-8') as f:
    num_records = sum(1 for line in f)

# determine the indices of the records to load
subset_size = int(num_records * 0.1)  # load 10% of the records
indices = random.sample(range(num_records), subset_size)

# load the random subset of records in chunks
chunks = []
json_reader = pd.read_json(file_path, lines=True, chunksize=chunk_size)
for chunk in json_reader:
    filtered_chunk = chunk[chunk.index.isin(indices)]
    if not filtered_chunk.empty:
        chunks.append(filtered_chunk)

# concatenate the chunks into a single DataFrame
df = pd.concat(chunks)


In [7]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
8,ZKvDG2sBvHVdF5oBNUOpAQ,wSTuiTk-sKNdcFyprzZAjg,B5XSoSG3SfvQGtKEGQ1tSQ,3,1,1,0,This easter instead of going to Lopez Lake we ...,2016-03-30 22:46:33
9,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
17,LnGZB0fjfgeVDVz5IHuEVA,j2wlzrntrbKwyOcOiB3l3w,rBdG_23USc7DletfZ11xGA,4,1,0,0,The hubby and I have been here on multiple occ...,2014-08-10 19:41:43
18,u2vzZaOqJ2feRshaaF1doQ,NDZvyYHTUWWu-kqgQzzDGQ,CLEWowfkj-wKYJlQDqT1aw,5,2,0,1,I go to blow bar to get my brows done by natal...,2016-03-07 00:02:18


In [8]:
df.loc[:,'stars'].value_counts(normalize=True)

5    0.462920
4    0.208488
1    0.152756
3    0.098199
2    0.077636
Name: stars, dtype: float64

In [9]:
yelp_reviews = df.loc[:, ['text','stars', 'useful','funny','cool']]

In [10]:
yelp_reviews.shape

(699028, 5)

In [11]:
yelp_reviews.head()

,text,stars,useful,funny,cool
0,"If you decide to eat here, just be aware it is...",3,0,0,0
8,This easter instead of going to Lopez Lake we ...,3,1,1,0
9,Had a party of 6 here for hibachi. Our waitres...,3,0,0,0
17,The hubby and I have been here on multiple occ...,4,1,0,0
18,I go to blow bar to get my brows done by natal...,5,2,0,1


In [12]:
# Tokenizing and lemmatizing: I had some issue switching to Eshan's branch.
# Please edit if I missed a step
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
#Pre-processing strings
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].str.lower()

In [14]:
#Removing punctuation
punc = r'[{}]+'.format(string.punctuation)
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: re.sub(punc,'',x))

In [15]:
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'] .apply(lambda x: nltk.word_tokenize(x))

In [16]:
stop_words = nltk.corpus.stopwords.words('english')
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: [i for i in x if i not in stop_words])

In [17]:
lemmatizer = nltk.stem.WordNetLemmatizer()
yelp_reviews.loc[:,'text'] = yelp_reviews.loc[:,'text'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

Model 1: Binary Logistic Regression

In [18]:
yelp_reviews.head()

,text,stars,useful,funny,cool
0,"[decide, eat, aware, going, take, 2, hour, beg...",3,0,0,0
8,"[easter, instead, going, lopez, lake, went, lo...",3,1,1,0
9,"[party, 6, hibachi, waitress, brought, separat...",3,0,0,0
17,"[hubby, multiple, occasion, loved, every, part...",4,1,0,0
18,"[go, blow, bar, get, brow, done, natalie, brow...",5,2,0,1


In [22]:
table_useful = pd.crosstab(yelp_reviews.loc[:,'useful'], yelp_reviews.loc[:,'stars'])
table_useful

stars,1,2,3,4,5
useful,,,,,
-1,1,0,1,0,0
0,49473,26561,36487,77507,194020
1,23517,12630,15599,33542,69016
2,12344,6276,7301,15027,27831
3,6980,3331,3680,7414,12870
...,...,...,...,...,...
332,1,0,0,0,0
357,0,0,0,0,1
371,0,0,0,0,1


Useful has 134 unique values. 91% of the data is restricted to values of useful < 5

In [24]:
yelp_reviews.loc[:,'useful'].value_counts(normalize=True)

0      0.549403
1      0.220741
2      0.098392
3      0.049032
4      0.026723
         ...   
273    0.000001
134    0.000001
171    0.000001
187    0.000001
325    0.000001
Name: useful, Length: 134, dtype: float64

In [25]:
table_funny = pd.crosstab(yelp_reviews.loc[:,'funny'], yelp_reviews.loc[:,'stars'])
table_funny

stars,1,2,3,4,5
funny,,,,,
0,83421,42718,55452,121122,286636
1,14263,7157,8229,15103,24689
2,4822,2169,2451,4338,5803
3,1918,945,980,1889,2350
4,891,470,501,1039,1293
...,...,...,...,...,...
278,0,0,1,0,0
313,0,0,0,0,1
326,0,1,0,0,0


In [26]:
yelp_reviews.loc[:,'funny'].value_counts(normalize=True)

0      0.843098
1      0.099339
2      0.028015
3      0.011562
4      0.006000
         ...   
70     0.000001
46     0.000001
116    0.000001
79     0.000001
90     0.000001
Name: funny, Length: 100, dtype: float64

84% of data is just the 0 label. 100 items but very little variance.

In [27]:
table_cool = pd.crosstab(yelp_reviews.loc[:,'cool'], yelp_reviews.loc[:,'stars'])
table_cool

stars,1,2,3,4,5
cool,,,,,
0,95703,46357,53673,100001,241661
1,8502,5514,9287,26795,51964
2,1617,1314,2818,8972,15126
3,473,481,1120,3671,5609
4,185,239,552,1918,2807
...,...,...,...,...,...
271,0,0,0,0,1
304,0,0,0,0,1
359,0,0,0,0,1


In [28]:
yelp_reviews.loc[:,'cool'].value_counts(normalize=True)

0      0.768775
1      0.146006
2      0.042698
3      0.016243
4      0.008156
         ...   
117    0.000001
80     0.000001
124    0.000001
113    0.000001
142    0.000001
Name: cool, Length: 119, dtype: float64

76% of data is label 0 and 14% is label 1.

**Logistic Regression- Part 1**
Creating two classes:
    labels 1-3 = bad (0)
    labels 4-5 = good (1)

In [29]:
# Making stars dichotomous with threshold of 3
yelp_reviews.loc[:, 'starsv1'] = 1
yelp_reviews.loc[yelp_reviews.loc[:,'stars'] < 4, 'starsv1'] = 0

In [30]:
yelp_reviews.head()

,text,stars,useful,funny,cool,starsv1
0,"[decide, eat, aware, going, take, 2, hour, beg...",3,0,0,0,0
8,"[easter, instead, going, lopez, lake, went, lo...",3,1,1,0,0
9,"[party, 6, hibachi, waitress, brought, separat...",3,0,0,0,0
17,"[hubby, multiple, occasion, loved, every, part...",4,1,0,0,1
18,"[go, blow, bar, get, brow, done, natalie, brow...",5,2,0,1,1


In [44]:
dict_pos = {}
dict_neg = {}
for list,star in zip(yelp_reviews.loc[:,'text'], yelp_reviews.loc[:,'starsv1']):
    if  star == 1:
        for word in list:
            if word not in dict_pos:
                dict_pos[word] = 0
            dict_pos[word] += 1
    else:
        for word in list:
            if word not in dict_neg:
                dict_neg[word] = 0
            dict_neg[word] += 1




In [62]:
def cumulative_count(list_tokens):
    dict_tot = {'pos' : 0, 'neg' : 0}
    for word in list_tokens:
        if word in dict_pos:
            dict_tot['pos'] += dict_pos[word]
        if word in dict_neg:
            dict_tot['neg'] += dict_neg[word]
    return dict_tot

In [63]:
yelp_reviews.loc[:,'cum_count'] = yelp_reviews.loc[:,'text'].apply(lambda x: cumulative_count(x))

In [64]:
yelp_reviews.head()

,text,stars,useful,funny,cool,starsv1,dict_pos,dict_neg,cum_count
0,"[decide, eat, aware, going, take, 2, hour, beg...",3,0,0,0,0,None,None,"{'pos': 1869867, 'neg': 1416199}"
8,"[easter, instead, going, lopez, lake, went, lo...",3,1,1,0,0,None,None,"{'pos': 1972119, 'neg': 1383590}"
9,"[party, 6, hibachi, waitress, brought, separat...",3,0,0,0,0,None,None,"{'pos': 1916005, 'neg': 1382503}"
17,"[hubby, multiple, occasion, loved, every, part...",4,1,0,0,1,"{'hubby': 1, 'multiple': 1, 'occasion': 1, 'lo...",None,"{'pos': 2113310, 'neg': 1334808}"
18,"[go, blow, bar, get, brow, done, natalie, brow...",5,2,0,1,1,"{'go': 1, 'blow': 2, 'bar': 1, 'get': 1, 'brow...",None,"{'pos': 1802551, 'neg': 944659}"
